In [1]:
import numpy as np
import random
import tensorflow as tf

In [2]:
def generate():
    global num_digit
    x1_d = np.random.random_integers(0,5*(10**(num_digit-1)))
    x2_d = np.random.random_integers(0,5*(10**(num_digit-1)))
    y_d = x1_d + x2_d
    x1_s = '00000' + str(x1_d)
    x2_s = '00000' + str(x2_d)
    y_s = '00000' + str(y_d)
    x1 = []
    x2 = []
    y = []  
    X = []
    
    for i in range(-num_digit,0):
        x1.append(int(x1_s[i]))
        x2.append(int(x2_s[i]))
        y.append(int(y_s[i]))
        X.append([int(x1_s[i]),int(x2_s[i])])
    X = np.array(X).T 
    y = np.array(y)
    return X[:,::-1],y[::-1],x1_d,x2_d,y_d

def generateb():
    global num_digit
    x1_d = np.random.random_integers(0,5*(10**(num_digit-1)))
    x2_d = np.random.random_integers(0,5*(10**(num_digit-1)))
    y_d = x1_d + x2_d
    x1_s = '00000' + str(x1_d)
    x2_s = '00000' + str(x2_d)
    y_s = '00000' + str(y_d)
    x1 = []
    x2 = []
    y = []  
    X = []
    
    for i in range(-num_digit,0):
        x1.append(int(x1_s[i]))
        x2.append(int(x2_s[i]))
        y.append(int(y_s[i]))
        X.append([int(x1_s[i]),int(x2_s[i])])
    X = np.array(X).T[:,::-1]
    X[1,:] += 10
    y = np.array(y)
    XX = np.zeros([20,num_digit])
    
    for i in range(num_digit):
        XX[X[:,i],i]=1    
    return XX,y[::-1],x1_d,x2_d,y_d

def generateany(num):
    
    x1_d = np.random.random_integers(0,99999)
    x2_d = np.random.random_integers(0,99999)
    y_d = x1_d + x2_d
    x1_s = '00000' + str(x1_d)
    x2_s = '00000' + str(x2_d)
    y_s = '00000' + str(y_d)
    x1 = []
    x2 = []
    y = []  
    X = []
    #num = np.random.random_integers(5,5)
    for i in range(-num,0):
        x1.append(int(x1_s[i]))
        x2.append(int(x2_s[i]))
        y.append(int(y_s[i]))
        X.append([int(x1_s[i]),int(x2_s[i])])
    X = np.array(X).T[:,::-1]
    X[1,:] += 10
    y = np.array(y)
    XX = np.zeros([20,num])
    
    for i in range(num):
        XX[X[:,i],i]=1    
    return XX,y[::-1],x1_d,x2_d,y_d

def generatef(x1_d,x2_d):
    global num_digit
   
    y_d = x1_d + x2_d
    x1_s = '00000' + str(x1_d)
    x2_s = '00000' + str(x2_d)
    y_s = '00000' + str(y_d)
    x1 = []
    x2 = []
    y = []  
    X = []
    
    for i in range(-num_digit,0):
        x1.append(int(x1_s[i]))
        x2.append(int(x2_s[i]))
        y.append(int(y_s[i]))
        X.append([int(x1_s[i]),int(x2_s[i])])
    X = np.array(X).T[:,::-1]
    X[1,:] += 10
    y = np.array(y)
    XX = np.zeros([20,num_digit])
    
    for i in range(num_digit):
        XX[X[:,i],i]=1    
    return XX,y[::-1],x1_d,x2_d,y_d

def generatet(num_digit,num_sample):
    array = []
    label = []
    for i in range(num_sample):
        X,Y,x1,x2,yo = generateany(num_digit)
        array.append(X)
        label.append(Y)
    array = np.array(array)
    label = np.array(label)
    array = np.rollaxis(array,2,0)
    return array,label

def softmax(out):
    a = np.exp(out)
    return a/np.sum(a,axis = 0, keepdims=True)
    
def sigmoid(activatee):
    ex = np.exp(activatee)
    return ex/(ex+1)

def sigmoid2(activatee):
    ex = np.exp(-activatee)
    return 1/(ex+1)

def sigmoid_prime(activated):
    return activated * (1-activated)

def relu(activatee):
    a = np.array(activatee)
    a[a<0] = 0
    return a

def relu_prime(activated):
    a = np.array(activated)
    a[a>=0] = 1
    a[a<0] = 0
    return a

def tanh(activatee):
    
    return np.tanh(activatee)

def tanh_prime(activated):
    a = 1-(activated)**2
    return a

def linear(activatee):
    return 0.1*activatee

def linear_prime(activated):
    return 0.1*np.ones_like(activated)

def arctan(activatee):
    return np.arctan(activatee)

def arctan_prime(activated):
    return 1/(activated**2+1)

In [3]:
def forward(X,Y,U,V,W,bh,bo):
    global func
    num_digit = X.shape[1]
    hidden = np.zeros([num_hidden,num_digit+1])
    output = np.zeros([10,num_digit])
    
    for t in range(num_digit):
        hidden[:,[t]] = eval(func)(U.dot(X[:,[t]]) + W.dot(hidden[:,[t-1]]) + bh)
        output[:,[t]] = V.dot(hidden[:,[t]] ) + bo
    prob = softmax(output)
    totalloss = np.mean(-np.log(prob[Y,range(num_digit)]))
    y = prob.argmax(axis=0)
    
    return hidden, output, prob, totalloss, y

def backward(X,Y,U,V,W,bh,bo,hidden,output,prob,totalloss,y):
    global func
    num_digit = X.shape[1]
    dW = np.zeros_like(W)
    dV = np.zeros_like(V)
    dU = np.zeros_like(U)
    dbh = np.zeros_like(bh)
    dbo = np.zeros_like(bo)
    
    dLdy = np.array(prob[:,:].T)
    dLdy[range(num_digit),Y] -= 1
    
    dLda = dLdy  # num_digit*10

    dLdV = (dLda.T.dot(hidden[:,:num_digit].T)).T # num_hidden*10
    
    dV += dLdV.T # 10*num_hidden
    dbo += np.sum(dLda,axis = 0, keepdims=True).T
    
    for t in range(num_digit-1,-1,-1):
        
        dLtdht = dLda[[t],:].dot(V) # 1*num_hidden
        dhtdzt = np.eye(num_hidden) * eval(func+'_prime')(hidden[:,t]) # num_hidden*num_hidden
        
        dLtdzt = dLtdht.dot(dhtdzt)        
        dLtdW = (dLtdzt.T.dot(hidden[:,[t-1]].T)).T # num_hidden*num_hidden
        dW += dLtdW.T # num_hidden*num_hidden
        dbh += dLtdzt.T
        
        dLtdU = (dLtdzt.T.dot(X[:,[t]].T)).T # 2*num_hidden
        dU += dLtdU.T # num_hidden*2
        
        dztdzj = np.eye(num_hidden)
        
        for j in range(t-1,-1,-1):
            
            dhjdzj = np.eye(num_hidden) * eval(func+'_prime')(hidden[:,j]) 
            
            dztdzj = dztdzj.dot((W.dot(dhjdzj))) # num_hidden*num_hidden
            
            dLtdzj = dLtdzt.dot(dztdzj) # 1*num_hidden
            dLtdW = (dLtdzj.T.dot(hidden[:,[j-1]].T)).T # num_hidden*num_hidden
            dW += dLtdW.T # num_hidden*num_hidden
            dbh += dLtdzj.T
            
            dLtdU = (dLtdzj.T.dot(X[:,[j]].T)).T # 2*num_hidden
            dU += dLtdU.T # num_hidden*2
        
    return dU,dW,dV,dbh,dbo    

In [30]:
num_hidden = 80
num_digit = 4
num_sample = 10000
input_dim = 20
epoch = 100
func = 'sigmoid'
batch = 10000

lr=0.05

U = (np.random.rand(num_hidden,input_dim))
W = ( np.random.rand(num_hidden,num_hidden))
V =  (np.random.rand(10,num_hidden))
Bh = (np.random.rand(num_hidden,1))
Bo = (np.random.rand(10,1))

losses = []
for e in range(epoch):
    sample=[]
    for i1 in range(num_sample):
        sample.append(generateany(num_digit))
    
    
    #random.shuffle(sample)
    loss = 0
    for b in range(batch):
        dW = np.zeros_like(W)
        dV = np.zeros_like(V)
        dU = np.zeros_like(U)
        dBh = np.zeros_like(Bh)
        dBo = np.zeros_like(Bo)
        
        batchsize = num_sample/batch
        start = int(b*batchsize)
        end = int(start + batchsize)
        
        for i in range(start,end):
            X,Y,x1,x2,yo = sample[i]
            hidden, output, prob, totalloss, y = forward(X,Y,U,V,W,Bh,Bo)
            
            #np.clip(output, -200, 200, out=output)
            du,dw,dv,dbh,dbo = backward(X,Y,U,V,W,Bh,Bo,hidden,output,prob,totalloss,y)
            dW += dw
            dU += du
            dV += dv
            dBh += dbh
            dBo += dbo
            
            #for dparam in [dW,dU,dV,dBh,dBo]:
                #np.clip(dparam, -5, 5, out=dparam)
            #print(hidden)
            #print(prob)
            #print(np.max(V),np.min(V))
            #print(np.max(W))
            #print(np.max(U))
            #print(dU)
            loss += totalloss 
        divide = 1
        U -= dU * lr/divide
        W -= dW * lr/divide
        V -= dV * lr/divide
        Bh -= dBh * lr/divide
        Bo -= dBo * lr/divide
        
    meanloss = loss/num_sample
    losses.append(meanloss)
    print('epoch %s,batch %s: '%(e,b),meanloss)
    print(x1,x2,Y[::-1],y[::-1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: DeprecationWarning: This function is deprecated. Please call randint(0, 99999 + 1) instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 99999 + 1) instead


epoch 0,batch 999:  4.41054367061
82800 72039 [4 8 3 9] [4 4 4 8]
epoch 1,batch 999:  2.31087132928
27786 40379 [8 1 6 5] [9 9 9 9]
epoch 2,batch 999:  1.97891466203
19401 24718 [4 1 1 9] [3 1 1 9]
epoch 3,batch 999:  1.09285717208
79716 71210 [0 9 2 6] [1 9 3 6]


KeyboardInterrupt: 

In [8]:
wrong = 0
for i in range(len(sample)):
    X,Y,x1,x2,yo = sample[i]
    hidden, output, prob, totalloss, y = forward(X,Y,U,V,W,Bh,Bo)
    if ~((Y==y).all()):
        wrong+=1
        
        
print(wrong)

0


In [12]:
num_digit = 2
for i in range(5000):    
    X,Y,x1,x2,yo = generateb()
    hidden, output, prob, totalloss, y = forward(X,Y,U,V,W,Bh,Bo)
    if ~((Y==y).all()):
        print(x1,x2,Y[::-1],y[::-1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: This function is deprecated. Please call randint(0, 50 + 1) instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: This function is deprecated. Please call randint(0, 50 + 1) instead


In [239]:

tf.reset_default_graph()

epoch_num = 100
sample_size = 10000
batch_size = 2000
iter_num=50
num_step = 1
rnn_size = 80



rnn = tf.contrib.rnn.BasicRNNCell(rnn_size,activation=tf.sigmoid)
initial_state =  tf.zeros([batch_size, rnn.state_size])

#input and label
X = tf.placeholder(tf.float32,shape=[num_step,batch_size,20])
Y = tf.placeholder(tf.int32,shape=[num_step,batch_size])


W = tf.Variable(tf.random_uniform([rnn_size,10],-1,1))
b = tf.Variable(tf.random_uniform([10],-1,1)+0) 

total_loss = 0
for step in range(num_step):
    if step == 0:
        hidden_state = initial_state
    output,hidden_state = rnn(X[step],hidden_state)
    logits = tf.matmul(output, W) + b
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = Y[step], logits = logits)
    total_loss += loss
LOSS = tf.reduce_mean(total_loss)/num_step

    
session =tf.Session()
session.run(tf.global_variables_initializer())
print(session.graph.get_collection('trainable_variables')[2].eval(session=session)
for e in range(epoch_num):
    xx,yy = generatet(num_step,sample_size)
    yy=yy.T
    for i in range(sample_size//batch_size):
        x = xx[:,i:i+batch_size,:]
        y = yy[:,i:i+batch_size]

        current_loss = session.run(LOSS,
                                  feed_dict={X:x,Y:y})
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(LOSS)
        session.run(train_step,feed_dict={X:x,Y:y})
    print(e,':',current_loss)

SyntaxError: invalid syntax (<ipython-input-239-fc4d4ec3b7f3>, line 38)

In [96]:
#input and label
x=[]
y=[]
for i in range(epoch_num):
    xx,yy = generatet(num_step,sample_size)
    yy=yy.T
    x.append(xx)
    y.append(yy)
x=np.array(x)
y=np.array(y)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: DeprecationWarning: This function is deprecated. Please call randint(0, 99999 + 1) instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 99999 + 1) instead


In [104]:
tf.reset_default_graph()

epoch_num = 20
sample_size = 10000
batch_size = 1
num_step = 4
rnn_size = 80



rnn = tf.contrib.rnn.BasicLSTMCell(rnn_size,activation=tf.tanh)


#input and label

#X = tf.placeholder(tf.float32,shape=[num_step,batch_size,20])
#Y = tf.placeholder(tf.int32,shape=[num_step,batch_size])

X = tf.constant(x,dtype=tf.float32)
Y = tf.constant(y)

W = tf.Variable(tf.random_normal([rnn_size,10]))
b = tf.Variable(tf.random_normal([10])) 

S = tf.placeholder(tf.int32,shape=())
E = tf.placeholder(tf.int32,shape=())

ep = tf.placeholder(tf.int32,shape=())
initial_state = rnn.zero_state(batch_size, dtype=tf.float32)
#initial_c = rnn.zero_state(batch_size, dtype=tf.float32)
output,state = tf.nn.dynamic_rnn(rnn,X[ep][:,S:E,:],
                                 initial_state=initial_state,
                                time_major=True)
logits = tf.tensordot(output,W,[[2],[0]])+b
prob = tf.nn.softmax(logits)
loss = tf.losses.sparse_softmax_cross_entropy(labels = Y[ep][:,S:E], logits = logits)
LOSS = tf.reduce_sum(loss)
LOSS_ = tf.reduce_mean(loss)
      

session =tf.Session()
session.run(tf.global_variables_initializer())
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(LOSS)
for epoch in range(epoch_num):
    
    for i in range(sample_size//batch_size):
        s = i * batch_size
        e = s + batch_size
        
        #'''
        
        
        a = session.run(train_step,feed_dict={S:s,E:e,ep:epoch})
        current_loss = session.run(LOSS_,
                          feed_dict={S:s,E:e,ep:epoch})
        #'''
    print(epoch,':',current_loss)


0 : 1.01291
1 : 0.0937216
2 : 0.0202048
3 : 0.0272923
4 : 0.00631827
5 : 0.00984098
6 : 0.0049605
7 : 0.00522065
8 : 0.00493579


KeyboardInterrupt: 

In [101]:
session.close()